# Emotion Classification in short texts with BERT

Applying BERT to the problem of multiclass text classification. Our dataset consists of written dialogs, messages and short stories. Each dialog utterance/message is labeled with one of the five emotion categories: joy, anger, sadness, fear, neutral.

## Workflow:
1. Import Data
2. Data preprocessing and downloading BERT
3. Training and validation
4. Saving the model


In [1]:
!pip install transformers datasets accelerate huggingface_hub



In [ ]:
import pandas as pd  # Load and manipulate data as tables
from datasets import Dataset  # Hugging Face Datasets - for model training
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score

In [3]:
import pandas as pd  # 👈 Add this line
from datasets import Dataset
# ✅ STEP 1: Load CSVs
train_df = pd.read_csv('/content/data_test.csv', encoding='utf-8')
test_df  = pd.read_csv('/content/data_test.csv', encoding='utf-8')

# ✅ STEP 2: Map emotion labels to integers — THIS IS REQUIRED TO FIX THE ERROR
label2id = {'joy': 0, 'sadness': 1, 'fear': 2, 'anger': 3, 'neutral': 4}
id2label = {v: k for k, v in label2id.items()}

train_df['labels'] = train_df['Emotion'].map(label2id)  # 👈 converts to integer
test_df['labels']  = test_df['Emotion'].map(label2id)

# Optional: Drop original label column
train_df.drop(columns=['Emotion'], inplace=True)
test_df.drop(columns=['Emotion'], inplace=True)

# ✅ STEP 3: Convert to Hugging Face Datasets
train_ds = Dataset.from_pandas(train_df)
test_ds  = Dataset.from_pandas(test_df)

In [4]:
# ✅ STEP 4: Tokenize the text
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_fn(examples):
    return tokenizer(
        examples['Text'],
        padding='max_length',   # 👈 important
        truncation=True,        # 👈 important
        max_length=350
    )

train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds  = test_ds.map(tokenize_fn, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3393 [00:00<?, ? examples/s]

Map:   0%|          | 0/3393 [00:00<?, ? examples/s]

In [5]:
# ✅ STEP 5: Set correct format for PyTorch training — OTHERWISE `Trainer` WILL FAIL
train_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


In [6]:
# ✅ STEP 6: Define model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# ✅ STEP 7: Set training args
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='bert_emotion',
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_dir='logs',
    eval_steps=len(train_ds)
)


In [8]:
# ✅ STEP 8: Define Trainer
from transformers import Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics_fn(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted')
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_fn
)

<ipython-input-8-09fc852d9ee2>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
# ✅ STEP 9: Train and evaluate
trainer.train()
trainer.evaluate()
# 4a43823d0fa4eabbbf9c0747c63f3776c43e351e

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pokemaster1502 (pokemaster1502-manav-rachna-educational-institutions) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.945500
1000,0.452400
1500,0.263900


{'eval_loss': 0.1550215184688568,
 'eval_accuracy': 0.9578544061302682,
 'eval_f1': 0.9579061642701947,
 'eval_runtime': 71.3591,
 'eval_samples_per_second': 47.548,
 'eval_steps_per_second': 7.932,
 'epoch': 3.0}

In [10]:
# ✅ STEP 10: Save
trainer.save_model('models/bert_emotion')
tokenizer.save_pretrained('models/bert_emotion')


('models/bert_emotion/tokenizer_config.json',
 'models/bert_emotion/special_tokens_map.json',
 'models/bert_emotion/vocab.txt',
 'models/bert_emotion/added_tokens.json',
 'models/bert_emotion/tokenizer.json')

In [22]:
# ✅ STEP 11: Inference
from transformers import pipeline

classifier = pipeline('text-classification', model='models/bert_emotion', tokenizer=tokenizer)
print(classifier("i woke up in the middle of the night"))
# www.kaggle.com/datasets/yashgupta1161/emotion-dataset

Device set to use cuda:0


[{'label': 'fear', 'score': 0.9916747212409973}]


In [14]:
import gradio as gr  # GUI interface for ML models

# Inference function to use in Gradio
def predict_emotion(text):
    """
    This function receives a text input, sends it through the emotion classifier,
    and returns the predicted emotion with score.
    """
    result = classifier(text)[0]  # classifier returns a list of dicts; we use the first
    label = result['label']
    score = round(result['score'], 4)
    return f"Emotion: {label} (Confidence: {score})"

# Create Gradio Interface
interface = gr.Interface(
    fn=predict_emotion,                    # function to call on input
    inputs=gr.Textbox(lines=3, label="Enter a sentence"),
    outputs=gr.Textbox(label="Predicted Emotion"),
    title="Emotion Classifier (BERT-based)",
    description="Type in a sentence and the model will classify it as one of: joy, sadness, fear, anger, or neutral."
)

# Launch the UI
interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://678e71d8dce94fa85c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
from sklearn.metrics import classification_report

# Get true and predicted labels
predictions = trainer.predict(test_ds)
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(axis=1)

# Print detailed per-class metrics
print("\n📄 Classification Report (per emotion class):")
print(classification_report(y_true, y_pred, target_names=list(label2id.keys())))



📄 Classification Report (per emotion class):
              precision    recall  f1-score   support

         joy       0.96      0.95      0.96       707
     sadness       0.96      0.96      0.96       676
        fear       0.97      0.97      0.97       679
       anger       0.96      0.96      0.96       693
     neutral       0.92      0.95      0.94       638

    accuracy                           0.96      3393
   macro avg       0.96      0.96      0.96      3393
weighted avg       0.96      0.96      0.96      3393

